# qtestNN

`qtest` is part of a Delaunay Triangulation algorithm.
This notebook explores how a neural network can be used to emulate its functionality.
The approach is to use `qtest` to build training and testing datasets.
Then additional intermediate values can be included in the NN to observe the improvement in accuracy.

I found that the circumcircle center cannot be determined by the net,
but all other intermediate values can be modeled by it.

## Preliminaries

Import the main packages.
Control tensorflow logging.

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
from random import random
from time import time

In [3]:
tf.logging.set_verbosity(tf.logging.ERROR)

## qtest

`qtest` in `pointset` operates on vertices.
This implementation operates on 4 points: the first three define the circumcircle,
the fourth is tested for inclusion in the circumcircle.
I've also broken down the steps into individual functions
so that meaningful intermediate values can be reproduced.

In [4]:
def dxdy(ix, iy, hx, hy):
    """The difference between two vectors"""
    return ix - hx, iy - hy

def norm(bx, by, cx, cy):
    """The norm used in calculating the circumcircle center"""
    return 2 * ( bx * cy - by * cx )

def circlexy(bx, by, cx, cy, d):
    """Return the circumcircle of the vectors and the origin"""
    ux = ( cy * (bx * bx + by * by) - by * (cx * cx + cy * cy) )
    uy = ( bx * (cx * cx + cy * cy) - cx * (bx * bx + by * by) )
    ux = ux / d
    uy = uy / d
    return ux, uy

def dist(x, y):
    """Return the distance between a point and the origin"""
    return int( (x * x + y * y)**0.5 * 100000) / 100000

def qtestxy(hx, hy, ix, iy, jx, jy, kx, ky):
    """Given 3 points, return True if the 4th point is not inside their circumcircle"""

    bx, by = dxdy(ix, iy, hx, hy)
    cx, cy = dxdy(jx, jy, hx, hy)
                                                                                                       
    n = norm(bx, by, cx, cy)

    ux, uy = circlexy(bx,  by, cx, cy, n)
    r = dist(ux, uy)

    x, y = dxdy(ux, uy, -hx, -hx)
    
    dx, dy = dxdy(kx, ky, x, y)
    d = dist(dx, dy)
    return True if d >= r else False

Check the function for a few cases.

In [5]:
print('.45: {}'.format(qtestxy(.4, .4, .5, .5, .6, .4, .5, .45)))
print('.49: {}'.format(qtestxy(.4, .4, .5, .5, .6, .4, .5, .49)))
print('.50: {}'.format(qtestxy(.4, .4, .5, .5, .6, .4, .5, .50)))
print('.51: {}'.format(qtestxy(.4, .4, .5, .5, .6, .4, .5, .51)))
print('.95: {}'.format(qtestxy(.4, .4, .5, .5, .6, .4, .5, .95)))

.45: False
.49: False
.50: False
.51: True
.95: True


## train

It's important to use `np` arrays for these structures so that the dimension of the tensors can be determined by tensorflow.

In [6]:
def train(epochs, ntrain, ntest, steps, layers, end, style=0, start=0):
    """train a data set, returning accuracy"""
    accuracy = []
    
    def samples(n):
        """returns an 8 vector of 4 random points"""
        res = []
        for _ in range(n):
            s = [ int(random()*100000)/100000 for _ in range(8) ]
            res.append(s)
        return np.array(res)

    trainingxs = samples(ntrain)
    testingxs = samples(ntest)

    def ysfor(xs):
        """return truth values using qtest"""
        return np.array([1 if qtestxy(h0, h1, i0, i1, j0, j1, k0, k1) else 0
                         for h0, h1, i0, i1, j0, j1, k0, k1 in xs])

    trainingys = ysfor(trainingxs)
    testingys = ysfor(testingxs)

    def parts0(hx, hy, ix, iy, jx, jy, kx, ky, pstlye=0):
        """return an array of intermediate calculations used in qtest"""

        bx, by = dxdy(ix, iy, hx, hy)
        cx, cy = dxdy(jx, jy, hx, hy)

                                                                                                        
        n = norm(bx, by, cx, cy)

        ux, uy = circlexy(bx,  by, cx, cy, n)
        r = dist(ux, uy)

        x, y = dxdy(ux, uy, -hx, -hx)
    
        dx, dy = dxdy(kx, ky, x, y)
        d = dist(dx, dy)
        v = 1.0 if d >= r else 0.0
        return [ bx, by, cx, cy, n, ux, uy, r, x, y, dx, dy, d, v ]
    
    def parts1(hx, hy, ix, iy, jx, jy, kx, ky):
        """return an array of distance attributes that might help find an accurate solution"""
        dx, dy = dxdy(kx, ky, hx, hy)
        kh = dist(dx, dy)
        
        dx, dy = dxdy(kx, ky, ix, iy)
        ki = dist(dx, dy)

        dx, dy = dxdy(kx, ky, jx, jy)
        kj = dist(dx, dy)

        dx, dy = dxdy(jx, jy, hx, hy)
        jh = dist(dx, dy)

        dx, dy = dxdy(jx, jy, ix, iy)
        ji = dist(dx, dy)

        dx, dy = dxdy(ix, iy, hx, hy)
        ih = dist(dx, dy)

        return [ kh, ki, kj, jh, ji, ih ]

    def parts2(hx, hy, ix, iy, jx, jy, kx, ky):
        """return an array of angle attributes that might help find an accurate solution"""

        def cos_angle(ax, ay, bx, by, cx, cy):
            # c2 = a2 + b2 - 2ab cos theta
            dx, dy = dxdy(ax, ay, bx, by)
            ab = dist(dx, dy)
            dx, dy = dxdy(cx, cy, bx, by)
            cb = dist(dx, dy)
            dx, dy = dxdy(ax, ay, cx, cy)
            ac = dist(dx, dy)
            return (ab*ab + cb*cb - ac*ac) / (2*ab*cb)

        hki = cos_angle(hx, hy, kx, ky, ix, iy)
        hkj = cos_angle(hx, hy, kx, ky, jx, jy)
        jki = cos_angle(jx, jy, kx, ky, ix, iy)

        hji = cos_angle(hx, hy, jx, jy, ix, iy)
        hjk = cos_angle(hx, hy, jx, jy, kx, ky)
        ijk = cos_angle(ix, iy, jx, jy, kx, ky)
        
        hij = cos_angle(hx, hy, ix, iy, jx, jy)
        hik = cos_angle(hx, hy, ix, iy, kx, ky)
        jik = cos_angle(jx, jy, ix, iy, kx, ky)

        ihj = cos_angle(ix, iy, hx, hy, jx, jy)
        ihk = cos_angle(ix, iy, hx, hy, kx, ky)
        jhk = cos_angle(jx, jy, hx, hy, kx, ky)

        return [ hki, hkj, jki,  hji, hjk, ijk,  hij, hik, jik,  ihj, ihk, jhk ]
    
    parts = parts0
    if style == 1:
        parts = parts1
    elif style == 2:
        parts = parts2


    def mixin(xs, start, end):
        """Update inputs with addition intermediate values"""
        ns = []
        for hx, hy, ix, iy, jx, jy, kx, ky in xs:
            ns.append([hx, hy, ix, iy, jx, jy, kx, ky] + parts(hx, hy, ix, iy, jx, jy, kx, ky)[start:end])
        return np.array(ns)
    
    trainingxs = mixin(trainingxs, start, end)
    testingxs = mixin(testingxs, start, end)
    
    classifier = tf.contrib.learn.DNNClassifier(
        feature_columns=[ tf.contrib.layers.real_valued_column("", dimension = trainingxs.shape[1] ) ],
        hidden_units=layers,
        n_classes=2)

    stime = time()
    for epoch in range(epochs):
        classifier.fit(x=trainingxs, y=trainingys, steps=steps)
        accuracy_score = classifier.evaluate(x=testingxs, y=testingys, steps=1)["accuracy"]
        print(" {:5.3f}".format(accuracy_score), end='', flush=True)
        if epoch % 10 == 9:
            print(' {:03d} {:6.4f} hours'.format(1+epoch, (time()-stime)/3600))
        accuracy.append(accuracy_score)
        if accuracy_score > 0.99:
            break
    print('\n{:6.4f} hours'.format((time()-stime)/3600))
    
    return accuracy

For the first series of test, use a large 3 layer, 100 node neural net.
On each iteration, reduce the number of intermediate results to determine
which values are most helpful.
Since the intermediate values includes the answer, we will have a iteration with
accurate results. And since the final iteration doesn't include intermediate values,
we will have a baseline case as well.

In [7]:
for end in [14, 13, 12, 10, 8, 7, 5, 4, 0]:
    print('end: {}'.format(end))
    train(epochs=100, ntrain=10000, ntest=1000, steps=100, layers=[100, 100, 100], end=end, style=0)

end: 14
 0.998
0.0053 hours
end: 13
 0.784 0.873 0.920 0.942 0.954 0.967 0.974 0.983 0.984 0.984 010 0.0499 hours
 0.981 0.985 0.972 0.985 0.985 0.986 0.961 0.986 0.987 0.986 020 0.0957 hours
 0.986 0.987 0.987 0.986 0.986 0.914 0.987 0.985 0.985 0.985 030 0.1410 hours
 0.985 0.985 0.985 0.985 0.985 0.986 0.986 0.986 0.986 0.986 040 0.1864 hours
 0.986 0.988 0.988 0.988 0.988 0.988 0.988 0.988 0.988 0.988 050 0.2318 hours
 0.988 0.988 0.988 0.988 0.988 0.988 0.988 0.989 0.989 0.989 060 0.2769 hours
 0.989 0.989 0.989 0.989 0.989 0.989 0.989 0.989 0.989 0.989 070 0.3223 hours
 0.989 0.989 0.989 0.989 0.989 0.989 0.989 0.989 0.989 0.989 080 0.3686 hours
 0.989 0.989 0.989 0.989 0.989 0.989 0.989 0.989 0.989 0.989 090 0.4138 hours
 0.989 0.989 0.989 0.989 0.989 0.989 0.989 0.989 0.989 0.988 100 0.4600 hours

0.4600 hours
end: 12
 0.685 0.791 0.879 0.925 0.938 0.945 0.937 0.971 0.975 0.910 010 0.0453 hours
 0.974 0.940 0.926 0.973 0.979 0.977 0.979 0.979 0.979 0.906 020 0.1001 hours
 0.981

With 7 additional intermediate values the accuracy is high
and it drops significantly with 5 values.
Values 5 and 6 are ux, uy - the center of the circumcircle.

First, I'll check if a larger net might have with 5 intermediate values.

In [8]:
for end in [5]:
    print('end: {}'.format(end))
    train(epochs=100, ntrain=10000, ntest=1000, steps=100, layers=[100, 100, 100, 100, 100], end=end, style=0)

end: 5
 0.663 0.683 0.711 0.757 0.782 0.792 0.797 0.796 0.799 0.800 010 0.0585 hours
 0.797 0.798 0.796 0.802 0.810 0.811 0.812 0.818 0.822 0.831 020 0.1151 hours
 0.826 0.835 0.840 0.821 0.795 0.835 0.866 0.837 0.866 0.876 030 0.1717 hours
 0.864 0.856 0.872 0.868 0.775 0.878 0.880 0.876 0.887 0.886 040 0.2336 hours
 0.888 0.887 0.889 0.854 0.887 0.888 0.890 0.875 0.890 0.891 050 0.2932 hours
 0.893 0.894 0.896 0.893 0.896 0.894 0.897 0.894 0.894 0.897 060 0.3500 hours
 0.896 0.898 0.898 0.898 0.897 0.897 0.898 0.898 0.899 0.899 070 0.4079 hours
 0.899 0.899 0.899 0.899 0.899 0.899 0.899 0.900 0.900 0.899 080 0.4693 hours
 0.899 0.900 0.900 0.900 0.900 0.900 0.900 0.900 0.901 0.902 090 0.5278 hours
 0.902 0.902 0.901 0.901 0.901 0.901 0.901 0.901 0.901 0.901 100 0.5929 hours

0.5929 hours


There is an improvement, but it isn't very significant.
May be the net needs to be wider instead of deeper.

In [9]:
for end in [5]:
    print('end: {}'.format(end))
    train(epochs=100, ntrain=10000, ntest=1000, steps=100, layers=[300, 300, 300], end=end, style=0)

end: 5
 0.637 0.641 0.705 0.728 0.754 0.773 0.783 0.786 0.792 0.797 010 0.1006 hours
 0.799 0.801 0.803 0.803 0.808 0.809 0.809 0.811 0.815 0.817 020 0.1965 hours
 0.819 0.825 0.827 0.828 0.825 0.832 0.835 0.835 0.835 0.838 030 0.2924 hours
 0.839 0.838 0.843 0.849 0.852 0.852 0.851 0.839 0.859 0.859 040 0.3890 hours
 0.827 0.858 0.840 0.862 0.864 0.868 0.858 0.834 0.844 0.871 050 0.4894 hours
 0.873 0.875 0.810 0.877 0.876 0.877 0.868 0.874 0.869 0.874 060 0.5857 hours
 0.876 0.881 0.887 0.802 0.883 0.882 0.881 0.884 0.886 0.766 070 0.6888 hours
 0.881 0.886 0.889 0.890 0.889 0.769 0.886 0.895 0.889 0.894 080 0.7895 hours
 0.895 0.898 0.896 0.895 0.895 0.894 0.896 0.897 0.896 0.894 090 0.8871 hours
 0.894 0.894 0.897 0.895 0.898 0.896 0.894 0.894 0.896 0.894 100 0.9836 hours

0.9836 hours


A wider net provides a smaller improvement.
So it doesn't seem to be a limitation of the size of the net.

Now I'll try to use distance features of the points as intermediate values.

In [10]:
for end in [6]:
    print('end: {}'.format(end))
    train(epochs=100, ntrain=10000, ntest=1000, steps=100, layers=[100, 100, 100], end=end, style=1)

end: 6
 0.656 0.688 0.704 0.717 0.751 0.778 0.781 0.790 0.795 0.801 010 0.0447 hours
 0.806 0.807 0.809 0.811 0.809 0.811 0.808 0.812 0.807 0.809 020 0.0887 hours
 0.809 0.808 0.807 0.806 0.810 0.810 0.809 0.809 0.813 0.812 030 0.1329 hours
 0.811 0.811 0.808 0.814 0.810 0.817 0.811 0.817 0.822 0.814 040 0.1770 hours
 0.823 0.819 0.822 0.823 0.826 0.821 0.824 0.826 0.830 0.831 050 0.2208 hours
 0.833 0.837 0.840 0.840 0.836 0.843 0.844 0.845 0.847 0.839 060 0.2646 hours
 0.839 0.847 0.847 0.847 0.842 0.845 0.854 0.848 0.845 0.834 070 0.3085 hours
 0.850 0.839 0.847 0.851 0.849 0.853 0.838 0.850 0.843 0.854 080 0.3533 hours
 0.847 0.848 0.847 0.834 0.840 0.858 0.847 0.855 0.842 0.854 090 0.3979 hours
 0.857 0.852 0.850 0.841 0.861 0.857 0.850 0.844 0.853 0.861 100 0.4497 hours

0.4497 hours


No improvement with the distance features.

Now try angle features.

In [11]:
for end in [12]:
    print('end: {}'.format(end))
    train(epochs=100, ntrain=10000, ntest=1000, steps=100, layers=[100, 100, 100], end=end, style=2)

end: 12
 0.704 0.727 0.752 0.752 0.758 0.773 0.790 0.802 0.806 0.814 010 0.0489 hours
 0.816 0.819 0.822 0.824 0.829 0.836 0.837 0.832 0.835 0.836 020 0.0976 hours
 0.842 0.842 0.842 0.849 0.846 0.847 0.849 0.844 0.851 0.847 030 0.1436 hours
 0.854 0.856 0.859 0.863 0.861 0.866 0.867 0.866 0.862 0.865 040 0.1949 hours
 0.862 0.862 0.866 0.864 0.864 0.864 0.866 0.867 0.861 0.865 050 0.2444 hours
 0.865 0.866 0.863 0.865 0.866 0.866 0.865 0.865 0.864 0.865 060 0.2913 hours
 0.865 0.862 0.865 0.859 0.866 0.862 0.859 0.863 0.862 0.865 070 0.3410 hours
 0.863 0.864 0.850 0.866 0.865 0.849 0.867 0.867 0.866 0.867 080 0.3874 hours
 0.865 0.868 0.867 0.865 0.864 0.867 0.866 0.863 0.865 0.841 090 0.4383 hours
 0.867 0.863 0.867 0.863 0.865 0.865 0.864 0.868 0.866 0.868 100 0.4870 hours

0.4870 hours


No improvement with these either.
But I'll do a quick check to see if the shape of the net makes any difference.

In [12]:
for end in [12]:
    print('end: {}'.format(end))
    train(epochs=1, ntrain=10000, ntest=1000, steps=100, layers=[300], end=end, style=2)

end: 12
 0.690
0.0050 hours


Now I'll focus on the circumcircle center.
I limit the additional features to only include the circumcircle center.

In [13]:
for end in [7]:
    print('end: {}'.format(end))
    train(epochs=100, ntrain=10000, ntest=1000, steps=100, layers=[100, 100, 100], start=end-2, end=end, style=0)

end: 7
 0.695 0.720 0.840 0.862 0.894 0.902 0.928 0.936 0.946 0.976 010 0.0459 hours
 0.979 0.980 0.981 0.980 0.983 0.913 0.984 0.984 0.986 0.986 020 0.1679 hours
 0.984 0.849 0.987 0.989 0.987 0.985 0.988 0.989 0.989 0.989 030 0.2121 hours
 0.990
0.2172 hours


The converged quickly.
It seems that the circumcircle center is the key missing piece of information
that the net cannot determine on its own.

Now I'll see how small a net is needed with this expanded feature set.

In [14]:
for end in [7]:
    print('end: {}'.format(end))
    train(epochs=100, ntrain=10000, ntest=1000, steps=100, layers=[100], start=end-2, end=end, style=0)

end: 7
 0.672 0.675 0.686 0.704 0.744 0.788 0.827 0.849 0.867 0.879 010 0.0371 hours
 0.893 0.903 0.908 0.915 0.920 0.927 0.935 0.936 0.937 0.940 020 0.0736 hours
 0.943 0.948 0.949 0.951 0.953 0.952 0.955 0.956 0.958 0.961 030 0.1091 hours
 0.963 0.964 0.966 0.968 0.968 0.970 0.970 0.969 0.971 0.972 040 0.1444 hours
 0.972 0.972 0.973 0.972 0.972 0.972 0.972 0.972 0.971 0.971 050 0.1773 hours
 0.973 0.973 0.974 0.974 0.974 0.974 0.975 0.976 0.976 0.976 060 0.2085 hours
 0.976 0.977 0.977 0.977 0.977 0.977 0.978 0.979 0.979 0.979 070 0.2396 hours
 0.979 0.979 0.979 0.980 0.980 0.980 0.980 0.981 0.981 0.981 080 0.2716 hours
 0.982 0.982 0.982 0.982 0.982 0.982 0.982 0.982 0.982 0.982 090 0.3064 hours
 0.982 0.982 0.982 0.982 0.982 0.982 0.983 0.983 0.983 0.983 100 0.3376 hours

0.3376 hours


One lay works almost as well as 3 layers.

But now wide does it need to be?

In [15]:
for end in [7]:
    print('end: {}'.format(end))
    train(epochs=100, ntrain=10000, ntest=1000, steps=100, layers=[50], start=end-2, end=end, style=0)

end: 7
 0.667 0.672 0.673 0.686 0.701 0.725 0.768 0.829 0.861 0.886 010 0.0311 hours
 0.905 0.919 0.922 0.925 0.930 0.931 0.932 0.935 0.939 0.941 020 0.0627 hours
 0.947 0.950 0.950 0.955 0.956 0.958 0.961 0.962 0.964 0.963 030 0.0935 hours
 0.963 0.965 0.965 0.965 0.965 0.966 0.966 0.965 0.967 0.967 040 0.1245 hours
 0.968 0.968 0.970 0.971 0.971 0.973 0.972 0.972 0.973 0.974 050 0.1568 hours
 0.974 0.974 0.974 0.974 0.975 0.975 0.975 0.975 0.975 0.975 060 0.1887 hours
 0.974 0.974 0.974 0.975 0.975 0.975 0.975 0.975 0.975 0.976 070 0.2196 hours
 0.976 0.975 0.975 0.975 0.975 0.975 0.975 0.975 0.975 0.976 080 0.2530 hours
 0.976 0.976 0.976 0.976 0.976 0.976 0.976 0.976 0.976 0.976 090 0.2856 hours
 0.976 0.976 0.976 0.976 0.976 0.975 0.975 0.974 0.975 0.975 100 0.3170 hours

0.3170 hours


50 nodes works well ... may be fewer are needed?

In [16]:
for end in [7]:
    print('end: {}'.format(end))
    train(epochs=100, ntrain=10000, ntest=1000, steps=100, layers=[25], start=end-2, end=end, style=0)

end: 7
 0.674 0.677 0.678 0.683 0.695 0.707 0.728 0.749 0.767 0.778 010 0.0340 hours
 0.795 0.826 0.841 0.855 0.869 0.876 0.882 0.898 0.904 0.904 020 0.0651 hours
 0.913 0.913 0.916 0.919 0.921 0.919 0.924 0.927 0.928 0.931 030 0.0963 hours
 0.933 0.936 0.936 0.939 0.940 0.940 0.940 0.942 0.943 0.945 040 0.1270 hours
 0.948 0.947 0.948 0.949 0.949 0.955 0.955 0.958 0.960 0.961 050 0.1572 hours
 0.962 0.962 0.964 0.965 0.966 0.967 0.969 0.970 0.970 0.970 060 0.1872 hours
 0.971 0.972 0.972 0.972 0.972 0.973 0.973 0.973 0.973 0.973 070 0.2173 hours
 0.973 0.974 0.974 0.974 0.974 0.974 0.974 0.974 0.974 0.976 080 0.2476 hours
 0.977 0.976 0.975 0.975 0.975 0.975 0.975 0.975 0.975 0.975 090 0.2777 hours
 0.976 0.975 0.974 0.974 0.974 0.974 0.975 0.974 0.974 0.973 100 0.3084 hours

0.3084 hours


In [17]:
for end in [7]:
    print('end: {}'.format(end))
    train(epochs=100, ntrain=10000, ntest=1000, steps=100, layers=[15], start=end-2, end=end, style=0)

end: 7
 0.662 0.663 0.665 0.666 0.675 0.698 0.735 0.781 0.821 0.848 010 0.0312 hours
 0.871 0.881 0.893 0.901 0.906 0.917 0.922 0.927 0.936 0.940 020 0.0630 hours
 0.943 0.947 0.948 0.952 0.958 0.958 0.961 0.958 0.963 0.964 030 0.0948 hours
 0.964 0.964 0.964 0.965 0.966 0.963 0.968 0.967 0.967 0.967 040 0.1242 hours
 0.968 0.968 0.968 0.969 0.969 0.969 0.969 0.968 0.970 0.970 050 0.1537 hours
 0.971 0.971 0.971 0.972 0.972 0.972 0.973 0.973 0.973 0.972 060 0.1833 hours
 0.973 0.973 0.973 0.974 0.974 0.975 0.975 0.975 0.975 0.975 070 14.7273 hours
 0.976 0.977 0.977 0.977 0.978 0.978 0.979 0.979 0.979 0.979 080 14.7574 hours
 0.979 0.979 0.979 0.979 0.979 0.979 0.979 0.979 0.979 0.979 090 14.7892 hours
 0.980 0.980 0.979 0.979 0.979 0.979 0.980 0.980 0.980 0.980 100 14.8189 hours

14.8189 hours


In [18]:
for end in [7]:
    print('end: {}'.format(end))
    train(epochs=100, ntrain=10000, ntest=1000, steps=100, layers=[10], start=end-2, end=end, style=0)

end: 7
 0.665 0.664 0.665 0.668 0.677 0.697 0.719 0.735 0.758 0.775 010 0.0296 hours
 0.783 0.789 0.797 0.803 0.818 0.826 0.834 0.836 0.841 0.845 020 0.0584 hours
 0.855 0.856 0.864 0.870 0.873 0.876 0.882 0.885 0.894 0.893 030 0.0878 hours
 0.900 0.906 0.906 0.907 0.908 0.907 0.913 0.914 0.914 0.916 040 0.1167 hours
 0.916 0.921 0.919 0.923 0.922 0.925 0.925 0.928 0.934 0.936 050 0.1464 hours
 0.937 0.939 0.940 0.943 0.945 0.945 0.944 0.945 0.946 0.947 060 0.1752 hours
 0.947 0.949 0.951 0.953 0.955 0.954 0.954 0.953 0.954 0.954 070 0.2041 hours
 0.954 0.955 0.958 0.958 0.959 0.959 0.958 0.958 0.958 0.958 080 0.2330 hours
 0.958 0.957 0.958 0.958 0.958 0.958 0.958 0.958 0.959 0.959 090 0.2621 hours
 0.959 0.959 0.959 0.959 0.959 0.959 0.959 0.959 0.960 0.960 100 0.2909 hours

0.2909 hours


# Final Remarks

`qtest` is just one part of a complex algorithm for determining a Delaunay triangulation.
To train a net to emulate `qtest`, I created a perfectly correct, noiseless, training data set (as well as a test data set). I created a relatively large net and found that accuracy was limited to ~83%.
I tried a variety of additional features to improve the accuracy.
As expected it worked very well with lots of intermediate features.
But the accuracy dropped significantly when the circumcircle center was not included
in the intermediate features.
I did a number of simulations to confirm that the circle center was the critically
missing piece of information.
With the circle center, I was able to reduce the net size significantly.

This implies that the net cannot determine the circle center on its own.
It also suggests that other features, such as distances, can to modeled by the net.

Further work could look at creating a net that takes 3 points as input and
determine their circumcircle center.
Or confirming that other features can be readily modeled by a net.
